<a href="https://colab.research.google.com/github/Maximilianwte/GenAI-Course/blob/main/BDD_GenAI_Train_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
!pip install -qq git+https://github.com/huggingface/diffusers.git
!wget -q https://github.com/huggingface/diffusers/raw/main/examples/dreambooth/train_dreambooth_lora.py
%pip install -qq -U --pre triton
%pip install -qq accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers
!accelerate config default
import os
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import torch
from torch import autocast
import numpy as np

In [ ]:
MODEL_NAME = "runwayml/stable-diffusion-v1-5"
OUTPUT_DIR = "/content/Test_Model"
!mkdir -p $OUTPUT_DIR

In [ ]:
!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --instance_data_dir='/content/XX' \
  --instance_prompt='xyzobject' \
  --seed=1 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --num_class_images=0 \
  --sample_batch_size=2 \
  --max_train_steps=3000 \
  --save_min_steps=2000 \
  --save_interval=250

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(OUTPUT_DIR, torch_dtype=torch.float16).to("cuda")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
g_cuda = torch.Generator(device='cuda')

In [ ]:
prompt = "xyzobject"
negative_prompt = ""


seed = np.random.randint(1, 1000)
g_cuda.manual_seed(seed)
guidance_scale = 7
with autocast("cuda"), torch.inference_mode():
    images = pipe(prompt, negative_prompt=negative_prompt, num_images_per_prompt=1, height=512, width=512, num_inference_steps=60, guidance_scale=guidance_scale, generator=g_cuda).images

images[0]